# import def func

In [1]:
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

from JohnsonUtil import LoggerFactory
log = LoggerFactory.log
# log_level = LoggerFactory.DEBUG
# log_level = LoggerFactory.INFO
log_level = LoggerFactory.ERROR
log.setLevel(log_level)
# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

log = LoggerFactory.log

In [2]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20,orderby='percent'):
    col_src = [
        "name",'percent', "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    if orderby in df.columns:
        df=df.sort_values('percent', ascending=False)
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl=ct.Resample_LABELS_Days[resample]
    elif  resample=='w':
        #200
        dl=ct.Resample_LABELS_Days[resample]
    else:
        #300
        dl=ct.Resample_LABELS_Days[resample]
    filter='y'
#     import pdb;pdb.set_trace()
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        print(eval(f'lastpTDX_DF_{resample}.shape'))
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    # resample=resample
    # if resample=='d' or resample=='3d':
    #     dl='60'
    # elif  resample=='w':
    #     dl='200'
    # else:
    #     dl='300'
    if resample=='d' or resample=='3d':
        dl=ct.Resample_LABELS_Days[resample]
    elif  resample=='w':
        #200
        dl=ct.Resample_LABELS_Days[resample]
    else:
        #300
        dl=ct.Resample_LABELS_Days[resample]
        
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [3]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [4]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [5]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-08-16',
 code
 300612    宣亚国际
 Name: name, dtype: object)

In [6]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-08/18/25 16:05:15'

code
000078   2025-08-18 15:00:00
688327   2025-08-18 15:00:02
Name: ticktime, dtype: datetime64[ns]

65.22900356849034

#### 实时成交

In [7]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:G:\sina_MultiIndex_data.h5
['/all_30']
G:\sina_MultiIndex_data.h5


(5113848, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-08-18 14:58:40,74.56,75.5,73.75,73.62,71537144,74.56
2025-08-18 15:00:01,74.50,75.5,73.75,73.62,72444281,75.31
2025-08-18 15:00:01,74.50,75.5,73.75,73.62,72444281,74.25


1.0

总量:17329.71 亿


ticktime    2025-08-19 00:00:00
close                 74.815035
volume                     0.31
vol                       72.44
amount                     54.2
Name: 603019, dtype: object

'2025-08-18'

#### 读取实时成交个股

In [8]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5189, 5),
         volume  name
 code                
 300059    2.61  东方财富
 300803    0.94   指南针
 000063    0.86  中兴通讯
 300033    0.71   同花顺
 300502    0.70   新易盛
 300308    0.70  中际旭创
 600111    0.70  北方稀土
 601138    0.67  工业富联
 600580    0.66  卧龙电驱
 600030    0.65  中信证券
 688256    0.64   寒武纪
 601162    0.63  天风证券
 300476    0.57  胜宏科技
 300339    0.51  润和软件
 002104    0.51  恒宝股份
 600392    0.47  盛和资源
 300750    0.47  宁德时代
 601606    0.45  长城军工
 688981    0.45  中芯国际
 002837    0.44   英维克)

##### 合并实时成交量及SinaDM

In [9]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey!!!

In [10]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [11]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [12]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [13]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx :0 b1>:5197 it:0.3 lastpTDX_DF_3d not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-18 16:05:25,765] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4573): tdx Out:4 code:['600225', '600321']


tdxdataT:2.74
DLTDXE:2.74 

In [14]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
# t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lasth1d > ma51d and lasth2d > ma52d and lasth3d > ma53d and ma51d > ma201d and ma52d >ma202d and ma53d > ma203d and macd > 0 and macddif > macddea')
show_tdx_data(t3d_code,limit=10)

Count:517 225


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
835438,戈碧迦,30.02,0,2025-04-07,48.22,25.03,18.11,47.91,41.81,14.67,18.80,1.20
872808,曙光数创,29.91,0,2025-04-08,99.06,62.75,50.59,43.50,92.75,43.56,1.20,0.10
301226,祥明智能,20.05,10,2025-04-08,40.25,29.02,24.56,42.81,39.25,21.66,1.20,0.10
300153,科泰电源,20.03,73,2025-06-04,42.84,34.47,26.00,1.30,41.41,26.00,0.00,-1.10
300499,高澜股份,20.02,173,2025-04-08,30.94,20.81,16.19,43.00,30.58,14.50,1.10,0.10
688108,赛诺医疗,19.98,83,2025-04-08,41.47,14.65,9.71,64.50,40.72,8.65,59.69,12.00
301128,强瑞技术,19.98,10,2025-04-08,89.31,62.25,44.75,48.31,87.62,37.62,29.41,-0.90
300727,润禾材料,19.91,24,2025-04-08,43.62,30.70,24.80,31.50,42.22,22.66,22.59,0.10
300620,光库科技,18.30,50,2025-04-07,97.00,53.66,39.38,1.10,91.44,35.34,0.10,0.10


In [15]:
if '301038' in t3d_code.index:
    h5_3d.loc['301038',['lastdu4','boll','ral','fib','percmax','macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

In [16]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [17]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5197 it:0.2 lastpTDX_DF_w not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-18 16:05:32,473] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4573): tdx Out:2 code:['600321', '600297']


tdxdataT:1.43
DLTDXE:1.43 

In [18]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query('lastp2d > ma51d and lastp3d > ma51d and macd > 0 and macddif > macddea')
show_tdx_data(tw_code,limit=10)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:925 225


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
835438,戈碧迦,30.02,0,2025-01-10,48.22,22.59,14.67,86.19,41.81,14.86,24.7,-1.10
430476,海能技术,29.94,0,2025-01-10,36.12,18.94,9.37,68.62,35.59,8.76,1.2,-1.10
872808,曙光数创,29.91,0,2025-05-23,99.06,78.69,43.56,35.09,92.75,50.59,1.2,1.20
832662,方盛股份,22.06,0,2025-01-03,47.91,31.91,16.38,58.00,45.09,17.39,16.5,11.00
301165,锐捷网络,20.08,7,2024-12-06,81.88,63.25,39.34,28.09,80.44,33.88,1.2,23.41
300730,科创信息,20.06,172,2025-01-10,21.25,15.80,9.70,39.59,20.22,10.53,-1.1,0.00
301226,祥明智能,20.05,10,2025-01-03,40.25,33.22,19.77,31.00,39.25,20.86,0.0,1.20
300499,高澜股份,20.02,173,2024-12-06,30.94,23.52,13.26,39.50,30.58,15.35,22.7,16.50
688098,申联生物,20.02,35,2025-01-10,12.83,6.64,4.97,42.91,12.82,4.86,35.0,1.30


In [19]:
if '301038' in tw_code.index:
    h5_w.loc['301038',['macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

### resample M

In [20]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5197 it:0.2 lastpTDX_DF_m not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-18 16:05:37,839] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4573): tdx Out:2 code:['600321', '600297']


tdxdataT:1.42
DLTDXE:1.42 

In [21]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
#月线站上,3月反弹,2月新高
tm_code = h5_m.query('lastl1d > lastl2d > lastl3d and lasth1d > lastp2d > lastp3d and  ma51d > ma52d and macd > 0 and macddif > macddea')   
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code,limit=10)

Count:950 225


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
835438,戈碧迦,30.02,0,2024-08-30,48.22,23.06,9.88,100.38,41.81,10.91,1.20,-1.10
688189,南新制药,20.09,40,2024-06-28,16.19,7.50,5.22,1.20,15.38,5.22,1.30,-1.10
688519,南亚新材,20.03,20,2024-08-30,65.19,37.25,16.30,39.91,64.56,17.42,45.81,0.00
688098,申联生物,20.02,35,2024-08-30,12.83,5.92,3.63,84.19,12.82,3.76,1.20,0.00
300767,震安科技,20.00,55,2024-06-28,19.86,11.64,7.29,40.59,19.30,8.20,40.59,-1.10
688108,赛诺医疗,19.98,83,2024-07-31,41.47,11.05,7.27,194.25,40.72,7.51,1.20,-1.10
301123,奕东电子,19.97,32,2024-08-30,47.28,23.31,14.19,81.69,46.91,15.54,5.30,-1.10
300620,光库科技,18.30,50,2024-08-30,97.00,51.06,30.50,84.62,91.44,31.20,1.30,-1.10
301161,唯万密封,17.30,0,2024-08-30,45.38,21.91,13.46,71.31,44.09,13.56,0.00,-1.10


In [22]:
if '301038' in tm_code.index:
    h5_m.loc['301038',['lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]
    
# h5_m.loc['301038',['lasth1d','lastp2d','lastp3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]    

lasth1d     40.0
lastl1d    27.95
lastl2d    15.37
lastl3d    13.92
lastp1d    36.25
ma51d      22.28
ma52d       18.0
ma53d      14.59
ma54d      13.97
Name: 301038, dtype: object

### resample D

In [23]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)
h5_d.shape

initdx :0 b1>:5197 it:0.2 lastpTDX_DF_d not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-08-18 16:05:43,342] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4573): tdx Out:8 code:['600225', '600811']


tdxdataT:5.39
DLTDXE:5.39 

(5186, 226)

In [24]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')

# td_code = h5_d.query(f'{get_filter(filterIDX)}')
# td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0')
td_code = h5_d.query(f'lasth1d > lasth2d  and ral > 10 and fib > 0 and boll > 0  and percent > lastdu4 \
and close >high4  and (low > open*0.99 or (per1d > 4 and low >= lastp1d))')

show_tdx_data(td_code,limit=20)
# if len(td_code) < 30:
#     write_code_to_blk(td_code.sort_values('percent', ascending=False).index,'065',rewrite=True)

Count:88 226


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
832662,方盛股份,22.06,0,2025-05-23,47.91,34.03,24.31,0.00,45.09,23.45,32.00,0.1
301226,祥明智能,20.05,10,2025-06-03,40.25,30.27,26.41,1.30,39.25,24.61,18.91,10.7
688519,南亚新材,20.03,20,2025-05-29,65.19,50.62,39.66,41.09,64.56,33.22,1.30,0.0
688098,申联生物,20.02,35,2025-05-23,12.83,9.96,6.04,22.30,12.82,5.73,0.00,1.2
300499,高澜股份,20.02,173,2025-05-23,30.94,22.83,16.98,25.20,30.58,16.36,26.00,1.2
688603,天承科技,20.02,5,2025-05-28,96.75,72.62,41.31,36.69,95.25,40.09,-0.10,7.3
688195,腾景科技,20.00,19,2025-06-03,64.69,48.59,40.28,31.70,64.19,38.31,1.30,0.3
300767,震安科技,20.00,55,2025-05-23,19.86,16.88,12.49,21.09,19.30,10.02,-0.10,0.1
301128,强瑞技术,19.98,10,2025-05-30,89.31,68.56,47.78,45.19,87.62,44.75,19.30,-0.9


In [25]:
if '301038' in td_code.index:
    h5_d.loc['301038',['lastdu4','close','boll','ral','fib','percmax','macd','macddif','macddea','macdlast1','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

#### filter test

In [26]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
# h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
code_3f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index))

show_tdx_data(t3d_code.loc[code_3f],limit=10)

Count:24 225


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
835438,戈碧迦,30.02,0,2025-04-07,48.22,25.03,18.11,47.91,41.81,14.67,18.80,1.2
688108,赛诺医疗,19.98,83,2025-04-08,41.47,14.65,9.71,64.50,40.72,8.65,59.69,12.0
300620,光库科技,18.30,50,2025-04-07,97.00,53.66,39.38,1.10,91.44,35.34,0.10,0.1
002536,飞龙股份,10.03,259,2025-04-10,32.59,21.30,13.47,45.81,32.47,10.30,14.60,10.6
603757,大元泵业,10.02,26,2025-04-08,51.84,27.59,21.34,75.50,50.31,18.91,1.20,1.3
600737,中粮糖业,10.01,115,2025-05-23,15.27,10.21,9.18,47.00,15.27,9.18,1.30,0.1
300690,双一科技,6.95,34,2025-04-08,41.38,27.20,21.81,51.59,40.38,17.02,0.10,0.1
603219,富佳股份,6.27,15,2025-04-08,18.80,15.87,13.34,35.69,18.06,12.60,0.10,0.0
301397,溯联股份,5.57,6,2025-04-08,49.09,35.84,25.08,56.59,48.38,19.52,33.69,1.2


In [27]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
# td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
# show_tdx_data(td_code2,limit=10)


#### 多周期筛选

In [35]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  )
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
# code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
code_f =t3d_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()

if len(code_f) > 0:
    show_tdx_data(t3d_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:950
tw_code:925
t3d_code:517
td_code:88
Count:24 225


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
835438,戈碧迦,30.02,0,2025-04-07,48.22,25.03,18.11,47.91,41.81,14.67,18.80,1.2
688108,赛诺医疗,19.98,83,2025-04-08,41.47,14.65,9.71,64.50,40.72,8.65,59.69,12.0
300620,光库科技,18.30,50,2025-04-07,97.00,53.66,39.38,1.10,91.44,35.34,0.10,0.1
002536,飞龙股份,10.03,259,2025-04-10,32.59,21.30,13.47,45.81,32.47,10.30,14.60,10.6
603757,大元泵业,10.02,26,2025-04-08,51.84,27.59,21.34,75.50,50.31,18.91,1.20,1.3
600737,中粮糖业,10.01,115,2025-05-23,15.27,10.21,9.18,47.00,15.27,9.18,1.30,0.1
300690,双一科技,6.95,34,2025-04-08,41.38,27.20,21.81,51.59,40.38,17.02,0.10,0.1
603219,富佳股份,6.27,15,2025-04-08,18.80,15.87,13.34,35.69,18.06,12.60,0.10,0.0
301397,溯联股份,5.57,6,2025-04-08,49.09,35.84,25.08,56.59,48.38,19.52,33.69,1.2


#### check data

In [36]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

False

24

In [38]:
show_tdx_data(t3d_code.loc[code_f])

Count:24 225


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
835438,戈碧迦,30.02,0,2025-04-07,48.22,25.03,18.11,47.91,41.81,14.67,18.80,1.2
688108,赛诺医疗,19.98,83,2025-04-08,41.47,14.65,9.71,64.50,40.72,8.65,59.69,12.0
300620,光库科技,18.30,50,2025-04-07,97.00,53.66,39.38,1.10,91.44,35.34,0.10,0.1
002536,飞龙股份,10.03,259,2025-04-10,32.59,21.30,13.47,45.81,32.47,10.30,14.60,10.6
603757,大元泵业,10.02,26,2025-04-08,51.84,27.59,21.34,75.50,50.31,18.91,1.20,1.3
600737,中粮糖业,10.01,115,2025-05-23,15.27,10.21,9.18,47.00,15.27,9.18,1.30,0.1
300690,双一科技,6.95,34,2025-04-08,41.38,27.20,21.81,51.59,40.38,17.02,0.10,0.1
603219,富佳股份,6.27,15,2025-04-08,18.80,15.87,13.34,35.69,18.06,12.60,0.10,0.0
301397,溯联股份,5.57,6,2025-04-08,49.09,35.84,25.08,56.59,48.38,19.52,33.69,1.2


### Write toblk

In [31]:
# write_code_to_blk(code_f,'077',rewrite=False)